### 212. Word Search II

$r$ is the number of rows.  
$c$ is the number of columns.  
$t$ is the maximum length of any word in the array words.  
$s$ is the sum of the lengths of all the words. 

### Backtracking (Trie)

**時間複雜度: $O(r * c * 4 * 3^{t-1} + s)$**  
**空間複雜度: $O(s)$**

In [1]:
from typing import List

# 定義 Trie（字典樹）節點結構
class TreeNode:
    def __init__(self):
        self.children = [None] * 26  # 子節點陣列（對應 26 個英文字母）
        self.ref = 0                 # 記錄有多少單詞經過此節點，用於剪枝
        self.idx = -1                # 若為單詞結尾，紀錄該單詞在 words 中的索引

    # 將單詞加入 Trie
    def addWord(self, word, word_idx):
        current = self
        current.ref += 1  # 根節點也算被引用一次
        for char in word:  # time: O(t)
            char_idx = ord(char) - ord('a')  # 將字母轉為索引 (a→0, b→1, ...)
            # 若對應子節點不存在，建立新節點
            if not current.children[char_idx]:
                current.children[char_idx] = TreeNode()
            current = current.children[char_idx]
            current.ref += 1  # 經過的每一層節點都增加引用次數
        current.idx = word_idx  # 將單詞結尾節點記錄其索引


class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        root = TreeNode()
        # 將所有單詞加入 Trie
        for i in range(len(words)):  # time: O(s)，最深為單詞長度 t，但相較於總節點數 s 通常可忽略
            root.addWord(word=words[i], word_idx=i)
        
        row_length = len(board)
        col_length = len(board[0])
        result = []  # 儲存找到的單詞

        # 深度優先搜尋 (DFS)
        def dfs(row, col, node):
            # 邊界條件與無效情況檢查
            if (
                row < 0 or col < 0 or
                row >= row_length or col >= col_length or
                board[row][col] == "*" or  # 已訪問過
                not node.children[ord(board[row][col]) - ord('a')]  # 字母不在 Trie 子節點中
            ):
                return
            
            char = board[row][col]               # 暫存目前字母
            char_idx = ord(char) - ord('a')      # 計算索引

            board[row][col] = "*"                # 標記為已訪問
            prev = node                          # 保存父節點
            node = node.children[char_idx]       # 進入下一層節點

            # 若此節點為某個單詞的結尾
            if node.idx != -1:
                result.append(words[node.idx])    # 將該單詞加入結果
                node.idx = -1                     # 避免重複加入 (加速)
                node.ref -= 1                     # 減少引用計數（已找到一個單詞）
                # 若該節點再無其他單詞引用，剪枝：刪除此分支 (加速)
                if node.ref == 0:
                    prev.children[char_idx] = None
                    board[row][col] = char        # 回溯：恢復原字母
                    return 

            # 向四個方向繼續搜尋
            # time: O(4)，第一次可以走上下左右 4 個方向
            # time: O(3^(t-1))，後續不能回到前一次的位置，只有三個方向
            dfs(row - 1, col, node)  # 上
            dfs(row + 1, col, node)  # 下
            dfs(row, col - 1, node)  # 左
            dfs(row, col + 1, node)  # 右

            # 回溯：恢復字母，讓其他路徑可使用
            board[row][col] = char

            return

        # 從每個棋盤格子作為起點進行 DFS 搜尋
        for row in range(row_length): # time: O(r)
            for col in range(col_length): # time: O(c)
                dfs(row, col, root)

        return result

In [2]:
board = [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]]
words = ["oa","oaa"]
# Output: ["eat","oath"]

Solution().findWords(board, words)

['oa', 'oaa']

#### Backtracking (Trie + Hash Set)

**時間複雜度: $O(r * c * 4 * 3^{t-1} + s)$**  
**空間複雜度: $O(s)$**

In [3]:
from typing import List

# 定義 Trie（字典樹）節點
class TrieNode():
    def __init__(self):
        self.children = {}  # 子節點，用字典儲存，例如 {'a': TrieNode, 'b': TrieNode, ...}
        self.end = False    # 標記此節點是否為單詞結尾

    # 將單詞加入 Trie
    def add_word(self, word):
        current = self
        # 逐字建立 Trie 結構
        for char in word:  # time: O(t)
            if char not in current.children:
                current.children[char] = TrieNode()
            current = current.children[char]
        current.end = True  # 標記單詞結尾

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 建立 Trie 並插入所有單詞
        root = TrieNode()
        for word in words: # time: O(s)，最深為單詞長度 t，但相較於總節點數 s 通常可忽略
            root.add_word(word)

        row_length = len(board)
        col_length = len(board[0])
            
        result = set()   # 使用集合避免重複單詞
        visit = set()    # 用來記錄 DFS 過程中已訪問的格子

        # 定義 DFS 回溯函式
        def dfs(row, col, node, word):
            # 邊界與無效情況檢查
            if (
                row < 0 or col < 0 or
                row >= row_length or col >= col_length or
                (row, col) in visit or
                board[row][col] not in node.children
            ):
                return
            
            # 將當前格子標記為已訪問
            visit.add((row, col))
            # 進入 Trie 的下一層
            node = node.children[board[row][col]]
            # 將當前字母加入組成中的單詞
            word += board[row][col]
            # 若到達單詞結尾，加入結果集合
            if node.end:
                result.add(word)

            # ---- DFS 四個方向擴展 ----
            # time: O(4)，第一次可以走上下左右 4 個方向
            # time: O(3^(t-1))，後續不能回到前一次的位置，只有三個方向
            dfs(row-1, col, node, word)
            dfs(row+1, col, node, word)
            dfs(row, col-1, node, word)
            dfs(row, col+1, node, word)

            # ---- 回溯步驟 ----
            visit.remove((row, col))  # 恢復狀態，讓其他路徑能重複使用該格

        # 從每一個格子作為起點進行 DFS 搜尋
        for row in range(row_length): # time: O(r)
            for col in range(col_length): # time: O(c)
                dfs(row, col, root, '')

        return list(result)

In [4]:
board = [["o","a","b","n"],["o","t","a","e"],["a","h","k","r"],["a","f","l","v"]]
words = ["oa","oaa"]
# Output: ["eat","oath"]

Solution().findWords(board, words)

['oa', 'oaa']

#### Backtracking

**時間複雜度: $O(r * c * 4^t + s) = O(r * c * 4^t)$**  
**空間複雜度: $O(t)$**

In [5]:
from typing import List

class Solution:
    def findWords(self, board: List[List[str]], words: List[str]) -> List[str]:
        # 取得棋盤的行數與列數
        row_length = len(board)
        col_length = len(board[0])
        # 儲存最終找到的單詞
        result_list = []

        # 定義 DFS 函式，用來檢查從 (row, col) 開始能否找到 word[index:]
        def dfs(word, index, row, col):
            # 如果已經走到單詞的最後一個字母之後，代表整個單詞找到
            if index == len(word):
                return True
            
            # 邊界檢查 & 是否匹配字母 & 是否已訪問過
            if (
                row < 0 or col < 0 or
                row >= row_length or col >= col_length or
                word[index] != board[row][col] or
                board[row][col] == '#'
            ):
                return False
            
            # 暫時標記為已訪問
            board[row][col] = '#'

            # 往上下左右四個方向繼續搜尋下一個字母 # time: O(4^t)
            result_flag = (
                dfs(word, index+1, row-1, col) or
                dfs(word, index+1, row+1, col) or
                dfs(word, index+1, row, col-1) or
                dfs(word, index+1, row, col+1)
            )

            # 回溯：恢復原本的字母，讓其他搜尋路徑可以使用
            board[row][col] = word[index]

            return result_flag

        # 針對每個單詞進行搜尋
        for word in words: #time: O(s)，+s 是在遍歷 words 陣列、讀取字元的過程中累積的總成本，不是 DFS 的搜尋成本
            flag = False  # 紀錄該單詞是否已經找到
            for row in range(row_length): # time: O(r)，以下才是 DFS 的成本
                if flag:  # 如果已經找到，提前跳出
                    break
                for col in range(col_length): # time: O(c)
                    # 如果首字母不符合，直接跳過
                    if board[row][col] != word[0]:
                        continue
                    # 如果能找到該單詞，加入結果
                    if dfs(word, 0, row, col):
                        result_list.append(word)
                        flag = True
                        break

        return result_list

In [6]:
board = [["o","a","a","n"],["e","t","a","e"],["i","h","k","r"],["i","f","l","v"]]
words = ["oath","pea","eat","rain"]
# Output: ["eat","oath"]

Solution().findWords(board, words)

['oath', 'eat']